In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))
from src.utils.seed import set_seed
from src.utils.config import Config
from src.preprocessing import get_data_loaders
from src.models import MLP
from src.uncertainty import MCDropoutEstimator, BayesianRanker

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [ ]:
config = Config('../configs/config.yaml')
train_loader, val_loader, test_loader, input_dim = get_data_loaders(config)

model = MLP(
    input_dim=input_dim,
    hidden_layers=config.model['mlp']['hidden_layers'],
    dropout=config.model['mlp']['dropout']
).to(device)

model.load_state_dict(torch.load('../reports/results/checkpoints/mlp_best.pth'))
print("✓ Loaded trained MLP model")

In [ ]:
mc_estimator = MCDropoutEstimator(
    model=model,
    n_samples=config.uncertainty['mc_dropout_samples'],
    device=device
)

print("Running MC Dropout inference...")

X_test_list = []
y_test_list = []

for X, y in test_loader:
    X_test_list.append(X)
    y_test_list.append(y)

X_test = torch.cat(X_test_list)
y_test = torch.cat(y_test_list)

mean_preds, var_preds = mc_estimator.predict(X_test, return_variance=True)

print("✓ Uncertainty estimation complete!")
print(f"Mean predictions shape: {mean_preds.shape}")
print(f"Variance shape: {var_preds.shape}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(mean_preds, bins=30, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Predicted Probability')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Prediction Distribution (MC Dropout)')
axes[0].grid(True, alpha=0.3)

axes[1].scatter(mean_preds, var_preds, alpha=0.5)
axes[1].set_xlabel('Predicted Probability')
axes[1].set_ylabel('Predictive Variance')
axes[1].set_title('Uncertainty vs Prediction')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
ranker = BayesianRanker(confidence_strength=10)

posterior_mean, lower_bound, upper_bound = ranker.compute_posterior(mean_preds, var_preds)
ranked_indices = ranker.rank(mean_preds, var_preds)

print("Top 10 ranked variants (with uncertainty):")
for i, idx in enumerate(ranked_indices[:10]):
    print(f"{i+1}. Index {idx}: Mean={posterior_mean[idx]:.4f}, CI=[{lower_bound[idx]:.4f}, {upper_bound[idx]:.4f}]")

In [ ]:
confidence_scores = ranker.confidence_score(mean_preds, var_preds)

plt.figure(figsize=(10, 5))
plt.hist(confidence_scores, bins=30, alpha=0.7, edgecolor='black')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')
plt.title('Confidence Score Distribution')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Mean confidence: {confidence_scores.mean():.4f}")
print(f"Std confidence: {confidence_scores.std():.4f}")

In [ ]:
print("✓ Uncertainty analysis complete!")